In [1]:
import sys, os
import pandas as pd
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam,SGD,RMSprop
from keras.regularizers import l2
from keras.utils import np_utils

df=pd.read_csv(r'E:\demo1\fer2013.csv')

X_train,train_y,X_test,test_y=[],[],[],[]
X_test1,test_y1=[],[]
#a,h,s=0,0,0
h1,h2=0,0
for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
            if (row['emotion']==0):
                train_y.append(row['emotion'])
                X_train.append(np.array(val,'float32'))#angry
            elif (row['emotion']==3):
                if(h1<5000):
                    train_y.append(1)#happy
                    X_train.append(np.array(val,'float32'))
                    h1+=1
            
            elif (row['emotion']==4):
                train_y.append(2)#sad
                X_train.append(np.array(val,'float32'))
            
            
        elif 'PublicTest' in row['Usage']:
            if (row['emotion']==0):
                test_y.append(row['emotion'])#angry
                X_test.append(np.array(val,'float32'))
            elif (row['emotion']==3):
                if(h2<700):
                    test_y.append(1)#happy
                    X_test.append(np.array(val,'float32'))
                    h2+=1
            elif (row['emotion']==4):
                test_y.append(2)#sad
                X_test.append(np.array(val,'float32'))
    except:
        print(f"error occured at index :{index} and row:{row}")
        
j=[0,0,0]
for i in train_y:
    j[i]+=1
print(j)
j=[0,0,0]
for i in test_y:
    j[i]+=1
print(j)
    

Using TensorFlow backend.


[3995, 5000, 4830]
[467, 700, 653]


In [2]:
num_features = 64
num_labels = 3
batch_size = 64
epochs = 100
width, height = 48, 48


X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)

#cannot produce
#normalizing data between oand 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [3]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 64)        36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 18, 18, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 9, 64)         

In [1]:
model.compile(loss=categorical_crossentropy,
              optimizer=SGD(lr=0.01),
              metrics=['accuracy'])
checkpoint = ModelCheckpoint(r"E:\demo1\ash.h5", monitor='val_accuracy',save_weights_only=True, mode='max', verbose=1,
                             save_best_only=True)


model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True,
         callbacks=[checkpoint])

fer_json = model.to_json()
with open("ash.json", "w") as json_file:
    json_file.write(fer_json)

NameError: name 'model' is not defined

In [5]:
#sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model1 = Sequential()
model1.add(Conv2D(32, (3, 3), input_shape=(X_train.shape[1:])))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model1.add(Conv2D(32, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model1.add(Conv2D(64, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model1.add(Flatten())
model1.add(Dense(64))
model1.add(Activation('relu'))
model1.add(Dropout(0.5))
model1.add(Dense(3))
model1.add(Activation('softmax'))

opt=RMSprop(learning_rate=0.01)

model1.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
checkpoint1 = ModelCheckpoint(r"E:\demo1\n1fer3.h5", monitor='val_accuracy',save_weights_only=True, mode='max', verbose=1,
                             save_best_only=True)


model1.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True,
         callbacks=[checkpoint1])

fer_json1 = model1.to_json()
with open(r"E:\demo1\n1fer3.json", "w") as json_file:
    json_file.write(fer_json1)

Train on 13825 samples, validate on 1820 samples
Epoch 1/100
13825/13825 [==============================] - 47s 3ms/step - loss: 1.1013 - accuracy: 0.3552 - val_loss: 1.0851 - val_accuracy: 0.3846

Epoch 00001: val_accuracy improved from -inf to 0.38462, saving model to E:\demo1\n1fer3.h5
Epoch 2/100
13825/13825 [==============================] - 48s 3ms/step - loss: 1.0949 - accuracy: 0.3664 - val_loss: 1.0918 - val_accuracy: 0.3846

Epoch 00002: val_accuracy did not improve from 0.38462
Epoch 3/100
13825/13825 [==============================] - 49s 4ms/step - loss: 1.0952 - accuracy: 0.3520 - val_loss: 1.0886 - val_accuracy: 0.3588

Epoch 00003: val_accuracy did not improve from 0.38462
Epoch 4/100
13825/13825 [==============================] - 50s 4ms/step - loss: 1.0951 - accuracy: 0.3601 - val_loss: 1.0876 - val_accuracy: 0.3588

Epoch 00004: val_accuracy did not improve from 0.38462
Epoch 5/100
13825/13825 [==============================] - 50s 4ms/step - loss: 1.0952 - accuracy:

KeyboardInterrupt: 

In [14]:
fer_json1 = model1.to_json()
with open(r"E:\demo1\n1fer3.json", "w") as json_file:
    json_file.write(fer_json1)